<a href="https://colab.research.google.com/github/Nerikoutchala/Nerikoutchala/blob/main/BigData_MLlib.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Big Data II - Prácticas**
### **MLlib**

**Por: Fco. Javier García Castellano**.

*Profesor Titular de Universidad. Departamento de Ciencias de Computación e Inteligencia Artificial (DECSAI). Universidad de Granada.*

## **ÍNDICE**


En este *notebook*: 
1. Empezaremos a conoce la parte de Machine Learning de Apache Spark:MLlib.
2. Veremos como preparar los datos para que un modelo pueda aprenderlos.
3. Aprenderemos a utilizar las tubería para optimizar el flujo de trabajo. 
4. Veremos como validar en Spark los modelos aprendidos

Contenidos:
1. Introducción. 

2. Preparar los DataFrame para Spark ML. 
  
3. Tuberías de Spark (Pipelines).

4. Tuberías con Machine Learning.

5. Métricas para la evaluación de los modelos de aprendizaje supervisado.

6. Validación de los modelos de aprendizaje supervisado.

7. Ajuste de los hiperparámetros de los modelos. 

8. Referencias Biobliográficas


##**1. INTRODUCCIÓN.**

MLlib es la biblioteca de Machine Learning de Apache Spark. Lo interesante de MLlib, es que los algoritmos de MLlib están pensados para entornos distribuidos, son altamente escalables y con tolerancia a fallos, pues están basados en Spark.

Proporciona las siguientes herramientas:

*   Algoritmos de Machine Learning: algoritmos comunes de clasificación, regresión, clustering o filtros colaborativos.
*   Herramientas de preprocesamiento de datos: herramienta para extraer, seleccionar o transformar las variables.
*   Tuberías (Pipelines): es una herramienta para optimizar los flujos de trabajo en Machine Learning.
*   Persistencia: para guardar y leer algoritmos, modelos y tuberías.
*   Utilidades:  álgebra lineal, estadística, manejo de datos, etc.


Cuando usamos Python, dentro de MLlib podemos encontrar dos interfaces de programación:
* [Spark ML](https://spark.apache.org/docs/latest/api/python/pyspark.ml.html ) (o [DataFrame MLlib API](https://spark.apache.org/docs/latest/mllib-guide.html#mllib-main-guide)): Que está basada en DataFrames y que está dentro del paquete `spark.ml`. Desde la versión 2.0 esta la versión por defecto y la que se aconseja usar. 
* [Spark MLlib](https://spark.apache.org/docs/latest/api/python/pyspark.mllib.html ) (o [RDD MLlib API](https://spark.apache.org/docs/latest/mllib-guide.html#mllib-rdd-based-api-guide)): Es la interfaz de programación original y está basada en RDD. Está dentro del paquete `spark.mllib`. Actualmente está en modo mantenimiento.

El cambio de pasar de API basada en RDDs a otra basada en DataFrames, se debe a que, como hemos visto, éstos últimos son más rápidos y fáciles de manejar.

En este cuaderno (notebook) vamos a seguir trabajando con Spark en Google Colab, por lo que los prolegómenos son los mismos, como se muestran a continuación.

In [1]:
#Primero instalamos Apache Spark con Hadoop
!wget -q http://www-eu.apache.org/dist/spark/spark-3.2.3/spark-3.2.3-bin-hadoop2.7.tgz
!tar xf spark-3.2.3-bin-hadoop2.7.tgz 

#Instalamos los paquetes de Python para trabajar con Spark
!pip install findspark #Instalamos FindSpark
!pip install pyspark   #Instalamos Spark

#Indicamos a PySpark donde está Spark
import findspark
findspark.init("spark-3.2.3-bin-hadoop2.7")#SPARK_HOME

#Inicializamos las variables de entorno
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/default-java" 
os.environ["SPARK_HOME"] = "/content/spark-3.2.3-bin-hadoop2.7"



#Creamos una sesión de Spark para poder trabajar
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("Ejemplo Machine Learning PySpark") \
    .getOrCreate()  

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 16.7 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824025 sha256=764ee534e65c77a15739bb50fcfc449df997ae312ece611b3c53ec936ad66199
  Stored in directory: /root/.cache/pip/wheels/6c/e3/9b/0525ce8a69478916513509d43693511463c6468db0de237c86
Successfully built pyspark


En un proyecto de Ciencia de Datos normalmente implica preprocesar los datos, selección de variables de entrada, aprendizaje de un modelo y evaluación de los resultados, lo que
denominamos el **ciclo de vida** de la Ciencia de Datos. Lo que vamos a ver en esta cápsula es como llevar a cabo dicho ciclo de vida mediante Apache Spark. 

##**2. PREPARAR LOS DATAFRAMES PARA SPARK ML.**

Para poder trabajar con los algoritmos de Machine Learning de Spark, tendremos que preparar los datos para que puedan ser procesados. Es más, los algoritmos de Machine Learning de Spark ML, trabajan sólo con **datos en formato numérico**. Por tanto, es esencial convertir a numérica, cualquier variable categórica que esté en nuestro conjunto datos. 

Veamos algunos de los métodos que se pueden usar para transformar variables:

* [`StringIndexer`](http://spark.apache.org/docs/latest/ml-features.html#stringindexer): Asigna un número entero a cada posible categoría. Se asigna el valor cero a la categoría más frecuente 1 a la siguiente y así sucesivamente. Tenemos que indicarle la columna que queremos indexar y el nombre de la columna de salida. En los metadatos se guarda el esquema usado por si queremos deshacer este paso con el método [`IndexToString()`](http://spark.apache.org/docs/latest/ml-features.html#indextostring).


* [`OneHotEncoderEstimator`](http://spark.apache.org/docs/latest/ml-features.html#onehotencoderestimator): Crea una columna para cada valor distinto que exista en el atributo que estamos codificando y, para cada instancia, marcar con un 1 la columna a la que pertenezca dicho registro y dejar las demás con 0.  En Spark no se puede pasar directamente una variable categórica a `OneHotEncoderEstimator` sino que tendremos que convertirla a índices previamente con `StringIndexer`. 

   <!--Esta codificación se usará en aquellos algoritmos que esperan atributos continuos, como por ejemplo la regresion logística. No obstante, la representación one-hot no tendremos que usarla en los atributos categóricos, después de haber usado StringIndexer, cuando vayamos a usar modelos que sí aceptan variables categóricas, como por eemplo, los árboles de decisión.-->

*  [`Vector Assembler`](http://spark.apache.org/docs/latest/ml-features.html#vectorassembler): Nos permite combinar una lista dada de columnas en una sola columna vector. Esto es debido a que los algoritmos de Machine Learning aceptan un DataFrame donde las variables de entrada están en una sola columna y, si se trata de aprendizaje supervisado, habrá además una segunda columna con la variable de salida. 

  Este paso se produce normalmente **al final del preprocesamiento de los datos**, cuando ya vamos a pasarle los datos de entrenamiento al método de Machine Learning. 


Para ilustrar los distintos ejemplos, vamos a trabajar con el mismo problema de clasificación: una versión reducida del problema [SUSY](https://archive.ics.uci.edu/ml/datasets/SUSY). Nos descargaremos los datos para trabajar con ellos. 

Posteriormente, convertiremos la variable de salida (clase) a entero. Después, uniremos las variables de entrada en una columna vector, usando para ello `vectorAssembler'. Finalmente, mostramos sólo estas dos columnas generadas, una con la variable de salida en formato numérico y otra con las variables de entrada agrupadas.


In [2]:
#Nos descargamos los ficheros de datos en Google Colab
!wget -nv --no-check-certificate 'https://docs.google.com/uc?export=download&id=1HOrM49tCLA_NqHyD_ps_cv483FPN7aWo' -O susy-10k-tra.csv
!wget -nv --no-check-certificate 'https://docs.google.com/uc?export=download&id=1HT80d5cwU7HMi2XK8CNgxgHvxRZEZB_d' -O susy-10k-tst.csv

#Leemos los conjuntos de entrenamiento y test
dfTra = spark.read.csv('susy-10k-tra.csv', inferSchema=True, header=True)
dfTst = spark.read.csv('susy-10k-tst.csv', inferSchema=True, header=True)

#Convertimos la variable clase en entera
dfTra=dfTra.withColumn("clase",dfTra.clase.cast("Integer"))
dfTst=dfTst.withColumn("clase",dfTst.clase.cast("Integer"))

#Preprocesamos los datos para ser utilizados en un algoritmo de ML
from pyspark.ml.feature import VectorAssembler

#Unimos las variables de entrada con VectorAssembler
assembler = VectorAssembler(inputCols=dfTra.columns[:-1],outputCol="atributos")
train = assembler.transform(dfTra)
train.select('atributos','clase').show(5)

#Uso el mismo VectorAssemble para el conjunto de test (las columnas son iguales)
test = assembler.transform(dfTst)
test.select('atributos','clase').show(5)


2023-03-14 16:34:34 URL:https://doc-08-bc-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/kqfqq5gt2d8je0nd0v246a49op17h2re/1678811625000/11180625338828972622/*/1HOrM49tCLA_NqHyD_ps_cv483FPN7aWo?e=download&uuid=9f8171e3-ee02-443d-bf7d-f69a4eba4945 [3463157/3463157] -> "susy-10k-tra.csv" [1]
2023-03-14 16:34:35 URL:https://doc-14-bc-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/hhts6dj2o1g8ccfah9gphopq23aor3fq/1678811625000/11180625338828972622/*/1HT80d5cwU7HMi2XK8CNgxgHvxRZEZB_d?e=download&uuid=a30b0228-e976-4bc6-89ee-e75db81761e5 [3469204/3469204] -> "susy-10k-tst.csv" [1]
+--------------------+-----+
|           atributos|clase|
+--------------------+-----+
|[0.64334195852279...|    1|
|[1.23746168613433...|    1|
|[1.34331297874450...|    0|
|[0.65310519933700...|    0|
|[1.55166482925415...|    1|
+--------------------+-----+
only showing top 5 rows

+--------------------+-----+
|           atributos|clase|
+--------------------+

##**3. TUBERÍAS DE SPARK (PIPELINES).**

Un proyecto de Ciencia de Datos normalmente implica preprocesar los datos (por ejemplo,  selección de variables de entrada, eliminación de ruido, balanceo de datos, preparar los datos para los algoritmos de aprendizaje,etc.) aprendizaje de un modelo y evaluación de los resultados. Por tanto, tenemos que realizar una **serie de transformaciones de los datos en secuencia**. 

Las [tuberías (pipelines)](https://spark.apache.org/docs/latest/ml-pipeline.html
 ) nos hacen más cómodo y optimizado combinar diferentes algoritmos en un sólo flujo de trabajo. Para trabajar con tuberías es necesario conocer algunos conceptos previos:

* DataFrame: La estructura de datos con la que vamos a trabajar y que ya deberíamos conocer.

* **[Transformador](http://spark.apache.org/docs/latest/ml-features.html)**: Un Transformador es un algoritmo que puede transformar un DataFrame en otro DataFrame. Por ejemplo, en la sección anterior hemos visto varias transformaciones para las variables categóricas. Otro ejemplo, sería un modelo Machine Learning que transforma un DataFrame con sólo variables de entrada en predicciones de la variable de salida. Un Transformador es una abstracción que incluye transformaciones sobre las variables o modelos aprendidos. Técnicamente, un Transformador implementa el método `transform()`, el cual convierte un DataFrame en otro, normalmente añadiendo una o más columnas.

* **Estimador**: Un Estimador es un algoritmo que puede ser ajustado a un DataFrame (o aprendido a partir de él) para producir un Transformador. Por ejemplo, un algoritmo de aprendizaje es un Estimador que se entrena (usando el método `fit()`) con un DataFrame y produce un modelo. 

* **Tuberías**: Una tubería encadena múltiples Transformadores y Estimadores para especificar un flujo de trabajo de Machine Learning. Por ejemplo, podríamos añadir la indexación y el ensamblado de las variables de entrada en una tubería y luego aprender una regresión logística.

* Parámetros: Todos los Transformadores y Estimadores comparten una interfaz común para especificar parámetros.


Una tubería es una secuencia de etapas, donde cada etapa es un Transformador o un Estimador. Estas etapas se ejecutan en orden, y el DataFrame de entrada es modificado en cada etapa. Para etapas de Transformadores, el método `transform()` es invocado sobre el DataFrame. Para etapas de Estimadores, el método `fit()` es invocado para producir un Transformador. El método `transform()` es llamado en dicho Transformador y el DataFrame para realizar la transformación.


Siguiendo el anterior ejemplo, podríamos haber hecho todas las transformaciones usando una tubería de la siguiente manera: 


In [3]:
#Unimos las variables de entrada con VectorAssembler
assembler = VectorAssembler(inputCols=dfTra.columns[:-1],outputCol="atributos")

#Añadimos el preprocesamiento a una tubería
from pyspark.ml import Pipeline
tuberiaParcial = Pipeline().setStages([assembler])

#Construimos el modelo
tuberia = tuberiaParcial.fit(dfTra)

#Transformamos los datos y los mostramos
datosPreprocesados = tuberia.transform(dfTra)
datosPreprocesados.select('atributos','clase').show(5)

+--------------------+-----+
|           atributos|clase|
+--------------------+-----+
|[0.64334195852279...|    1|
|[1.23746168613433...|    1|
|[1.34331297874450...|    0|
|[0.65310519933700...|    0|
|[1.55166482925415...|    1|
+--------------------+-----+
only showing top 5 rows



De esta forma, usando tuberías, se puede optimizar el preprocesamiento que se realizan sobre los datos, lo cual es muy importante cuando el volumen de datos es grande. 

##**4. TUBERÍAS CON MACHINE LEARNING.**

Como hemos visto, hemos incluido todo el preprocesamiento de nuestro conjunto de datos en una tubería. Ahora veremos como añadir también un algoritmo de Machine Learning. 

Seguiremos un ejemplo, concretamente de aprendizaje supervisado,  donde aprenderemos un clasificador de regresión logística a partir de los datos generados anteriormente.

Siguiendo la terminología de Spark aprenderemos un modelo Transformador (que posee el método `transform()`) mediante un Estimador de regresión logística (que posee el método `fit()`). Es decir, aprenderemos un Estimador con los datos de entrenamiento y, el modelo aprendido lo usaremos como un Transformador sobre los datos de test. Dicho Transformador tomará los datos de test y los convertirá en una serie de predicciones. Dichas predicciones las usaremos para calcular la precisión del modelo. 




In [4]:
from pyspark.ml.classification import LogisticRegression
#Aprendemos el Modelo de Regresión Logística
rl = LogisticRegression(featuresCol = 'atributos', labelCol = 'clase', maxIter=10)
rlModel = rl.fit(train)

#Obtenemos las predicciones sobre el conjunto de test
predicciones = rlModel.transform(test)

#Ya podemos mostrar la precisión del modelo
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluador = MulticlassClassificationEvaluator(labelCol="clase", metricName="accuracy")
print('Accuracy:', evaluador.evaluate(predicciones))

Accuracy: 0.8043


El anterior ejemplo, lo podemos mejorar. Ahora lo que haremos será añadir a la tubería el modelo de regresión logística, de esta forma conseguimos que Spark optimice más aún el proceso. 

Definiremos la tubería y aprenderemos el modelo. Dicho modelo, con el preprocesado de datos y el clasificador, se aplicará al conjunto de test para hacer las predicciones. Con estas predicciones podemos calcular la proporción de bien clasificados.  

In [5]:
#Unimos las variables de entrada con VectorAssembler
assembler = VectorAssembler(inputCols=dfTra.columns[:-1],outputCol="atributos")

#Incorporamos el modelo de regresión logística
rl = LogisticRegression(featuresCol = 'atributos', labelCol = 'clase', maxIter=10)

#Añadimos las etapas a una tubería
etapas=[assembler,rl]
modeloRL = Pipeline().setStages(etapas)

#Construimos el modelo: preprocesamiento + regresión logística
clasificador = modeloRL.fit(dfTra)

#Obtenemos las predicciones sobre el conjunto de test 
predicciones = clasificador.transform(dfTst)

#Ya podemos mostrar la precisión del modelo
evaluator = MulticlassClassificationEvaluator(labelCol="clase", metricName="accuracy")
print('Accuracy:', evaluator.evaluate(predicciones))


Accuracy: 0.8043


Veamos a continuación las métricas y modelos de validación que podemos usar en Spark, pero nos centraremos en el aprendizaje supervisado. 

##**5. MÉTRICAS PARA LA EVALUACIÓN DE LOS MODELOS DE APRENDIZAJE SUPERVISADO.**

Dentro de las herramientas de aprendizaje supervisado, tenemos clasificadores y regresores, dependiendo de si la variable que queremos predecir es discreta o continua, respectivamente.

En los clasificadores, distinguiremos aquellos que son para **clasificación binaria**, es decir, la variable de salida sólo tiene dos estados, de la **clasificación multiclase**, es decir, aquellos en los que la variable de salida tiene más de dos estados.

Tengamos en cuenta, que los clasificadores que nos permiten trabajar con problemas multiclase también nos permiten trabajar con problemas binarios. Lo cual tiene sentido, es decir,  si un algoritmo es capaz de discriminar entre 4 ó 10 valores de la variable de salida, también debería ser capaz de distinguir entre dos.

No obstante, también podemos usar clasificadores binarios en problemas multiclase, por ejemplo, utilizando la metodología Uno-contra-todos.  Este esquema construye un clasificador base para cada valor de la variable de salida y dicho clasificador distingue el valor i-ésimo del resto. 


Primero, nos vamos a centrar en el caso de los problemas de clasificación binaria. En estos clasificadores vamos a usar la clase `BinaryClassificationEvaluator` que nos permite calcular dos medidas:

1.   `areaUnderROC` : El área bajo la curva ROC, o también ROC AUC. 
2.   `areaUnderPR`: Es el área bajo la curva  Precision-Recall, o también Precision-Recall AUC. La curva PR es el resultado de dibujar la gráfica entre el precision y el recall.  

<!-- Esta gráfica nos permite ver a partir de qué recall tenemos una degradación de la precisión y viceversa. Lo ideal sería una curva que se acerque lo máximo posible a la esquina superior derecha (alta precisión y alto recall). Esta métrica es más apropiada para conjuntos no balanceados, es decir, hay un reparto desigual acusado entre las dos clases del problema.


Vamos a usar el ejemplo anterior para mostrar ambas métricas:

 #La regresión logística, también nos permite mostras la curvas ROC
import matplotlib.pyplot as plt
trainingSummary = clasificador.stages[-1].summary
roc = trainingSummary.roc.toPandas()
plt.plot(roc['FPR'].tolist(),roc['TPR'].tolist(),color='orange')
plt.ylabel('False Positive Rate')
plt.xlabel('True Positive Rate')
plt.title('ROC Curve')
plt.plot([0, 1], [0, 1], color='navy',  linestyle='--')
plt.show()
-->

In [6]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

#Ya podemos mostrar la bondad de las predicciones del modelo cn AUC
evaluator = BinaryClassificationEvaluator(labelCol="clase", metricName="areaUnderROC")
print('ROC AUC:', evaluator.evaluate(predicciones))
print ('areaUnderPR', evaluator.evaluate(predicciones, {evaluator.metricName: "areaUnderPR"}) )

ROC AUC: 0.8528537606105937
areaUnderPR 0.8012755689164685


Veamos ahora el caso, de la clasificación multiclase. Hay que señalar que las métricas que usan cuando hay más de una clase se pueden usar igualmente en clasificación binaria.  En este caso vamos a usar la clase `MulticlassClassificationEvaluator` y las medidas que podemos obtener son:
*  `accuracy`: También llamado proporción de bien clasificados. Se refiere a la ratio de ejemplos positivos correctamente identificados con respecto al total. 
*  `weightedPrecision`: Es la precisión promediada por etiqueta, es decir, se calcula la media de la precisión por cada etiqueta.
*  `weightedRecall`: Es el *Recall* promediado por etiqueta, es decir, se calcula la media de la precisión por cada etiqueta.
*  `f1`: También denominado *F-measure*, es la métrica que combina *precision* y  *recall*.

Sigamos con el ejemplo anterior que, aún siendo un problema de clasificación binario, podemos calcular estas medidas:


In [7]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

#Ya podemos mostrar la bondad de las predicciones del modelo multiclase
evaluator = MulticlassClassificationEvaluator(labelCol="clase")
print('F1:', evaluator.evaluate(predicciones))
print('Accuracy:', evaluator.evaluate(predicciones, {evaluator.metricName: "accuracy"}))
print('Weighted Precision:', evaluator.evaluate(predicciones, {evaluator.metricName: "weightedPrecision"}))
print('Weighted Recall:', evaluator.evaluate(predicciones, {evaluator.metricName: "weightedRecall"}))

F1: 0.8039722690022877
Accuracy: 0.8043
Weighted Precision: 0.803679034355548
Weighted Recall: 0.8043




Cuando la variable a predecir es una variable continua, se dice que el modelo Machine Learning es un regresor. Para regresión vamos a usar la clase `RegressionEvaluator` y las medidas vistas en el módulo 4 que podemos obtener son:
* `mse`: Error cuadrático medio. Mide el error cuadrado promedio de nuestras predicciones. Para cada punto, calcula la diferencia cuadrada entre las predicciones y el objetivo y luego promedia esos valores.
* `rmse`:  Es la raíz cuadrada del error cuadrático medio. La raíz cuadrada se introduce para hacer que la escala de los errores sea igual a la escala de los objetivos.

* `r2`:  El coeficiente de determinación, o $r^2$, está estrechamente relacionada con el error cuadrático medio, pero siempre estará entre -∞ y 1.
* `mae`: Error absoluto medio. El error se calcula como un promedio de diferencias absolutas entre los valores objetivo y las predicciones. De esta forma, todas las diferencias individuales se ponderan por igual en el promedio.


Sigamos con el ejemplo anterior, pero al SER un problema de clasificación binario, no van a tener mucho sentido los resultados. Sólo nos sirve para ver cómo se usa:
<!-- from pyspark.ml.evaluation import RegressionEvaluator

#Ya podemos mostrar la bondad de las predicciones del modelo multiclase
evaluator = RegressionEvaluator(labelCol="clase")
print('rmse:', evaluator.evaluate(predicciones))
print('mse:', evaluator.evaluate(predicciones, {evaluator.metricName: "mse"}))
print('r2:', evaluator.evaluate(predicciones, {evaluator.metricName: "r2"}))
print('mae:', evaluator.evaluate(predicciones, {evaluator.metricName: "mae"}))-->



In [8]:
from pyspark.ml.evaluation import RegressionEvaluator

#Ya podemos mostrar la bondad de las predicciones de un regresor 
evaluator = RegressionEvaluator(labelCol="clase")
print('rmse:', evaluator.evaluate(predicciones))
print('r2:', evaluator.evaluate(predicciones, {evaluator.metricName: "r2"}))


rmse: 0.4423799272118933
r2: 0.12360226428573085


##**6. VALIDACIÓN DE LOS MODELOS DE APRENDIZAJE SUPERVISADO.**

Hasta ahora, hemos validado nuestro modelo de ejemplo con diversas métricas, pero usando un sólo método: Entrenamiento y Test. Aunque ya teníamos los datos partidos, se puede automatizar este proceso a partir de un sólo conjunto de datos. Podemos usar la clase `TrainValidationSplit`, indicándole el modelo del clasificador, un objeto `ParamGrimBuilder` que se utiliza en otras funcionalidades que veremos más adelante, la clase que se encarga de la evaluación y finalmente la proporción de la partición. 


In [9]:
from pyspark.ml.tuning import TrainValidationSplit, ParamGridBuilder

#Uno los conjuntos de entrenamiento y test en un dataframe
df=dfTra.union(dfTst)

#Creamos una clase Evaluador
evaluador = MulticlassClassificationEvaluator(labelCol="clase", metricName="accuracy")

#Usamos la regresión logística para ser evaluada 
validadorTT= TrainValidationSplit(estimator=modeloRL,
                           estimatorParamMaps=ParamGridBuilder().build(),
                           evaluator=evaluador,
                           trainRatio=0.8) # 80% de los datos para entrenamiento
validadorTT.setSeed(2022)
#Entrenamos el modelo con el conjunto completo de datos
modeloTT=validadorTT.fit(df)

#Mostramos la precisión del modelo
print(modeloTT.getEvaluator().getMetricName(), ':',modeloTT.validationMetrics[0] )

accuracy : 0.78626524010948



Este tipo de validación es bastante sencillo y rápido, aunque tiene poco rigor estadístico y los resultados dependen demasiado de cómo hagamos la partición de los datos. A modo de curiosidad, si la semilla aleatoria elegida hubiese sido `seed=2020`, en lugar de `seed=2022`, los resultados en Accuracy hubieran sido un 1% mejores.

Al ser una validación tan sesgada, es preferible hacer una validación cruzada de, al menos, unas 10 hojas. No obstante, en Big Data, rara vez podremos permitirnos el lujo de escoger una validación tan costosa.

Para esta validación podemos usar la clase `CrossValidator`, y donde antes indicábamos el porcentaje de datos para entrenamiento, ahora indicaremos el número de hojas. 

In [10]:
from pyspark.ml.tuning import CrossValidator

#Validación cruzada
crossval = CrossValidator(estimator=modeloRL,
                          estimatorParamMaps=ParamGridBuilder().build(),
                          evaluator=evaluador,
                          numFolds=10) 
#Entrenamos el modelo
cvModel=crossval.fit(df)

#Ya podemos mostrar la precisión del modelo
print(cvModel.getEvaluator().getMetricName(), ':',cvModel.avgMetrics[0])

accuracy : 0.7915715430606218


##**7. AJUSTE DE LOS HIPERPARÁMETROS DE LOS MODELOS.**

Los algoritmos de aprendizaje automático aceptan una serie de parámetros que afectan a su funcionamiento. Estos parámetros de los modelos se denominan **hiperparámetros**. Por ejemplo, un número demasiado alto de árboles en un RandomForest puede hacer que tarde demasiado tiempo o que sobreajuste, mientras que un número demasiado bajo provocará una menor capacidad predictora. 

Por tanto, es útil encontrar los mejores valores de los hiperparámetros para que el modelo aprenda de una manera óptima el problema a resolver.

Aunque lo hemos visto en el apartado anterior, en realidad, Spark no proporciona ningún tipo de validación de los modelos. Hemos usado los dos tipos de validación existentes para el ajuste de los hiper parámetros.

En el ajuste de los hiperparámetros, tenemos que indicar que conjunto de valores pueden tomar cada parámetro usando la clase ` ParamGridBuilder`. Lo que hicimos en el apartado anterior era dejar esos conjuntos de valores vacíos y así tomaba sólo los que hubieramos indicado o sus valores por defecto.

Para hacer el ajuste de los hiperparámetros, vamos a indicar con dicha clase un conjunto de valores que puede tomar, por ejemplo, la regresión logística. 


In [11]:
from pyspark.ml.tuning import CrossValidator

# Usamos ParamGridBuilder para cosntruir un conjunto de parámetros sobre los que
# hacer una búsqueda. Con 3 valores para rl.maxIter y 2 valores para rl.regParam,
# tendremos 3 x 2 = 6 configuraciones de hiperparámetros distintas.
paramGrid = ParamGridBuilder() \
    .addGrid(rl.maxIter , [10, 100, 1000]) \
    .addGrid(rl.regParam, [0.1, 0.01]) \
    .build()


#Validación cruzada
crossval = CrossValidator(estimator=modeloRL,
                          estimatorParamMaps=paramGrid,
                          evaluator=evaluador,
                          numFolds=10) 

#Entrenamos el modelo con el conjunto de entrenamiento
cvModel=crossval.fit(dfTra)

#Mostramos las distintas evaluaciones calculadas. 
print(cvModel.getEvaluator().getMetricName(), ':',cvModel.avgMetrics)

#Utilizamos el conjunto de test para medir como de bueno es el modelo. Para 
# ello usamos la mejor configuración (bestModel) de los parámetros encontrada.
predicciones = cvModel.bestModel.transform(dfTst)

#Ya podemos mostrar la bondad del modelo
print('Accuracy:', evaluador.evaluate(predicciones))

accuracy : [0.7577256434658226, 0.7712002724930181, 0.7576249385312808, 0.7718680605128679, 0.7576249385312808, 0.7718680605128679]
Accuracy: 0.8035


Observemos un detalle del ejemplo anterior, para entrenar el modelo hemos usado el conjunto de entrenamiento. Para evaluar la bondad del modelo usamos el conjunto de test. Téngase en cuenta que podríamos tener un resultado sobreajustado si nos quedamos con el mejor resultado de la búsqueda. 

##**8.REFERENCIAS BIBLIOGRÁFICAS**

* The Apache Software Foundation. "Machine Learning Library (MLlib) Guide". (2020). [Acceso 9 de junio de 2020]. Disponible en: https://spark.apache.org/docs/latest/ml-guide.html

* The Apache Software Foundation. "Pyspark.ml package API". (2020). [Acceso 9 de junio de 2020]. Disponible en: https://spark.apache.org/docs/latest/api/python/pyspark.ml.html

* The Apache Software Foundation. "Extracting, transforming and selecting features". (2020). [Acceso 9 de junio de 2020]. Disponible en: http://spark.apache.org/docs/latest/ml-features.html

* The Apache Software Foundation. "ML Pipelines". (2020). [Acceso 9 de junio de 2020]. Disponible en: https://spark.apache.org/docs/latest/ml-pipeline.html



###**Referencias Adicionales**

* Lakshay Arora. "Want to Build Machine Learning Pipelines? A Quick Introduction using PySpark". (2019). [Acceso 9 de junio de 2020]. Disponible en: https://www.analyticsvidhya.com/blog/2019/11/build-machine-learning-pipelines-pyspark/

* Susan Li. "Machine Learning with PySpark and MLlib — Solving a Binary Classification Problem". (2018). [Acceso 9 de junio de 2020]. Disponible en:  https://towardsdatascience.com/machine-learning-with-pyspark-and-mllib-solving-a-binary-classification-problem-96396065d2aa

* The Apache Software Foundation. "Evaluation Metrics - RDD-based API". (2020). [Acceso 9 de junio de 2020]. Disponible en: https://spark.apache.org/docs/latest/mllib-evaluation-metrics.html


* The Apache Software Foundation. "ML Tuning: model selection and hyperparameter tuning". (2020). [Acceso 9 de junio de 2020]. Disponible en: https://spark.apache.org/docs/latest/ml-tuning.html#ml-tuning-model-selection-and-hyperparameter-tuning
